In [8]:
# Установка версий программных пакетов
#!pip install -qr ../requirements.txt

# Импортируем библиотеки
import numpy as np
#import pandas as pd

# графические библиотеки
from matplotlib import pyplot as plt
%matplotlib widget
import os

# Зафиксируем PYTHONHASHSEED для воспроиизводимости результатов обучения модели
seed_value = 0
os.environ['PYTHONHASHSEED'] = str(seed_value)

# Логгирование процесса
from comet_ml import Experiment

# библиотеки машинного обучения
import tensorflow as tf
import random

# библиотека взаимодействия с интерпретатором
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")


# Скрытие хода обучения модели,который загромождает ноутбук
from IPython.display import clear_output

# Библиотека вызова функций, специально разработанных для данного ноутбука
sys.path.insert(1, '../')
from utils.functions import config_reader, f1, callbacks, reset_random_seeds
from utils.data_reader import DataReader
from utils.data_loader import DataLoader
from utils import figures

# Импортируем константы из файла config
path_to_config='../config/data_config.json'
config = config_reader(path_to_config)

# Импортируем модели
from models.models import SimpleRNN_Model, LSTM_Model

# credentials = config_reader('../config/credentials.json')
# experiment = Experiment(
#     api_key = credentials.api_key, 
#     project_name = credentials.project_name,
#     workspace = credentials.workspace,
# );  
# experiment.set_name('model') # имя эксперимента

# константы и глобальные параметры данных
# Все исходные файлы размещены в папке data
PATH = config.PATH

# Папка для сохранения весов лучшей модели при обучении (исп-ся в ModelCheckpoint в функции callbacks)
PATH_TEMP_MODEL = config.PATH_TEMP_MODEL

if not os.path.exists(PATH_TEMP_MODEL):
    os.mkdir(PATH_TEMP_MODEL)    

# Папка для сохранения обученных моделей для последующего предсказания
PATH_FOR_MODEL = config.PATH_FOR_MODEL

if not os.path.exists(PATH_FOR_MODEL):
    os.mkdir(PATH_FOR_MODEL)

# 1. Установим начальное значение для генератора случайных чисел в Python
random.seed(seed_value)

# 2. Установим начальное значение для генератора случайных чисел в Numpy
np.random.seed(seed_value)

# 3. Установим начальное значение для генератора случайных чисел в tensorflow 
tf.random.set_seed(seed_value)

# 4. Конфигурация Tenzorflow
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

# Словарь для последующей агрегации данных. Изначально прописаны названия файлов в архиве
mounts = config.mounts.toDict()

print('Imports Done!', sep='\n\n')

### Read Data
#i = input('Enter pilot number, ex: 1, 2, 3, etc.')
i = str(2)

mounts[int(i)] = mounts.pop(i)
i = int(i)

# Загружаем данные в словарь с помощью DataLoader
data_for_nn = DataLoader(i, config)
mounts[i]['X_train_nn'] = data_for_nn.X_train_nn
mounts[i]['y_train_nn'] = data_for_nn.y_train_nn
mounts[i]['X_test_dataset'] = DataReader(
    os.path.join(config.PATH, config.mounts[str(i)].path_X_test_dataset)).data

print('Read Data Done!', sep='\n\n')

### Load and train SimpleRNN
X_train_nn = mounts[i]['X_train_nn']
y_train_nn = mounts[i]['y_train_nn']

# Создаем пустой лист для накопления данных с последующей корректировкой y_train
mounts[i]['y_trn_nn_ch_list'] = []

for splt_coef in range(10, 100, config.simpleRNN_delta_coef_splt): 
    
    # кол-во разных тренировок зависит от числа разбиений.
    val_splt_coef = splt_coef/100

    tf.keras.backend.clear_session()
    reset_random_seeds(seed_value) # сброс и задание random seed

    model = SimpleRNN_Model(X_train_nn, y_train_nn, 
                            units=config.simpleRNN_units).build_model()
    
    model = tf.keras.models.clone_model(model)
    
    model.compile(
        loss="mean_squared_error", 
        metrics=[f1], 
        optimizer='Adam', # по умолчанию learning rate=10e-3
    )

    history = model.fit(
        X_train_nn, 
        y_train_nn, 
        validation_split=val_splt_coef, 
        callbacks=callbacks(
            reduce_patience=config.reduce_patience, 
            stop_patience=config.stop_patience, 
            num_train=i,
            
            PATH_BEST_MODEL=config.PATH_TEMP_MODEL,
            monitor=config.ModelCheckpoint.monitor, 
            verbose=config.ModelCheckpoint.verbose, 
            mode=config.ModelCheckpoint.mode, 
            save_best_only=config.ModelCheckpoint.save_best_only,

            restore_best_weights=config.EarlyStopping.restore_best_weights,
            
            factor=config.ReduceLROnPlateau.factor, 
            min_lr=config.lr / config.ReduceLROnPlateau.min_lr_coeff),  # остальные параметры - смотри в functions.py
        epochs=config.simpleRNN_epochs, 
        verbose=config.simpleRNN_verbose
    )
    
    #experiment.log_parameter("callbacks", callbacks) # логгируем  callbacks
    
    y_pred_train_nn = model.predict(X_train_nn)  #m.predict(X_train_nn) 
    print(y_pred_train_nn.shape)
    
    #mounts[i]['model'] = model # модели сохранять не нужно
    
    mounts[i]['y_trn_nn_ch_list'].append(y_pred_train_nn)

# создаём переменную для записи среднего значения предсказаний
y_pred_train_nn = np.mean(mounts[i]['y_trn_nn_ch_list'], axis=0).argmax(axis=-1)
    
# сохраняем в словарь
mounts[i]['y_pred_train_nn'] = tf.keras.utils.to_categorical(y_pred_train_nn)

# Визуализируем ход обучения модели #1
#figures.plot_history(history, plot_counter='2-3')
#experiment.log_figure(figure_name='fig_2-3') # логгируем график


# # Отображение предсказания моделью SimpleRNN
# Pilot_id=3
# y_pred_train_nn = mount['y_pred_train_nn']
# y_pred_train_nn_mean = np.mean(x_trn_pred_dict[Pilot_id], axis=0)

# figures.get_gesture_prediction_plot(
#     Pilot_id=3, 
#     i=5, 
#     mounts=mounts, 
#     y_pred_train_nn_mean=y_pred_train_nn_mean, 
#     plot_counter='2-4'
# )

# #experiment.log_figure(figure_name='fig_2-4') # логгируем график

print('Load and train SimpleRNN Done!', sep='\n\n')

### Load and train LSTM
tf.keras.backend.clear_session()
reset_random_seeds(seed_value)
y_pred_train_nn = mounts[i]['y_pred_train_nn']

model_lstm = LSTM_Model(X_train_nn, y_pred_train_nn, 
                        lstm_units=config.lstm_units).build_model()    

model_lstm.compile(
    loss="categorical_crossentropy", 
    metrics=[f1], 
    optimizer='Adam', # по умолчанию learning rate=10e-3
)

# m_lstm = tf.keras.models.clone_model(model_lstm)

history2 = model_lstm.fit(
    X_train_nn,
    y_pred_train_nn,
    validation_split=config.lstm_val_splt,
    epochs=config.lstm_epochs,      
    verbose=config.lstm_verbose,
    callbacks=callbacks(
            reduce_patience=config.reduce_patience, 
            stop_patience=config.stop_patience, 
            
            PATH_BEST_MODEL=config.PATH_TEMP_MODEL,
            monitor=config.ModelCheckpoint.monitor, 
            verbose=config.ModelCheckpoint.verbose, 
            mode=config.ModelCheckpoint.mode, 
            save_best_only=config.ModelCheckpoint.save_best_only,

            restore_best_weights=config.EarlyStopping.restore_best_weights,
            
            factor=config.ReduceLROnPlateau.factor, 
            min_lr=config.lr / config.ReduceLROnPlateau.min_lr_coeff,
            num_train=i)  # остальные параметры - смотри в functions.py
    )

mounts[i]['history2'] = history2

mounts[i]['model_lstm'] = model_lstm

print('Load and train SimpleRNN Done!')

# сохранение обученной модели в папке по пути PATH_FOR_MODEL
model_lstm.save(os.path.join(PATH_FOR_MODEL, 'model_lstm_' + str(i) + '.h5'), save_format='h5')

print('Model LSTM saved!', sep='\n\n')

# Визуализируем ход обучения модели #2
#figures.plot_history(history2, plot_counter='2-5') 
#experiment.log_figure(figure_name='fig_2-5') # логгируем график

# # Визуализируем предскание жеста моделью
# Pilot_id=3 # номер пилота

# y_pred_train_nn_mean = mounts[Pilot_id]['y_pred_train_lstm']

# figures.get_gesture_prediction_plot(
#     Pilot_id=3, # номер пилота
#     i=18, # номер наблюдения
#     y_pred_train_nn_mean=y_pred_train_nn_mean, 
#     mounts=mounts, # словарь
#     plot_counter='2-6')

# #experiment.log_figure(figure_name='fig_2-6') # логгируем график

# Заканчиваем сессию логгирования эксперимента
#experiment.end()

# отображаем результат обучения
#experiment.display()



Imports Done!
Creating RawArray with float64 data, n_channels=50, n_times=23202
    Range : 0 ... 23201 =      0.000 ...   765.633 secs
Ready.
Creating RawArray with float64 data, n_channels=1, n_times=23202
    Range : 0 ... 23201 =      0.000 ...   765.633 secs
Ready.
Not setting metadata
264 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 264 events and 107 original time points ...
1 bad epochs dropped
Read Data Done!
input_shape = (None, 50) | output_units = 5
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, 50)]        0         
                                                                 
 batch_normalization (BatchN  (None, None, 50)         200       
 ormalization)                                                   
                                                    